# CPPM Explicit ions simulation

## Methodology

In this notebook we explore the interaction between lactoferrin and a β-lactoglobulin dimer using Metropolis-Hastings Monte Carlo (MC) simulations. The two protein structures are kept rigid and coarse grained so that each amino acid is represented by a single bead of size $\sigma_i$. These beads interact with a combined Lennard-Jones and Debye-Hückel potential to account for short-range repulsion and attraction, as well as electrostatic interactions in an aqueous electrolyte solution.
The ionization state of each amino acid is _fluctuating_ according to its p$K_a$-value and solution pH; this is done using a _constant pH ensemble_.
The MC system energy function is:

$$
U = \sum_i^{N-1}\sum_{j=i+1}^{N} \left \{ 4\epsilon_{ij} \left [ \left( \frac{\sigma_{ij}}{r_{ij}} \right )^{12}  - \left( \frac{\sigma_{ij}}{r_{ij}} \right )^{6}\right ]
  + k_BT \frac{\lambda_Bz_iz_j}{r_{ij}} \right \}
$$

where
$\lambda_B=7$ Å is the Bjerrum length,
$z$ are residue charges,
$r$ the inter-residue separation,
$\epsilon=0.05~k_BT$ the Lennard-Jones interaction strength.
The final term will be explaned below.

The mass centers of the two rotating proteins are fixed along the z-axis. For each mass center distance a simulation is performed where a virtual translation move is incorporated to get the force from with the potential of mean force is calculated. The proteins are allowed rotational MC moves and move attempts are also performed to (de-)protonate acidic and basic residues. The salt particles are allowed to both rotate and translate.
During simulation, we sample the radial distribution function, $g(R)$, which is related to the angularly averaged _potential of mean force_, $w(R) = -k_BT \ln g(R) + w_0$, where the reference state $w_0$ is chosen such that $w(R)\rightarrow 0$ for large $R$.

However, for the smaller distances, where to two proteins are placed closer to each other, were no or very few accepted moves performed, which resulted in unusable values of the virtual translation force. In order to come around this problem, parallel tempering was added to the simulation. Thus, this notebook was only used for explicit ion simulations with a protein-protein mass center distance of above 70 Å. This value was choosen by observing where too few moves were accepted in a simulation to produce a result and then an additional 5 Å were added to make sure that the offest distance was big enough to not cause any problems. Therefore, this notebook was used for mass cemter distances of 70 Å and above. 

Note that depending on how you structure your files you might need to make mainor changes to the script, also you need to define your work directory and directory of your fanus installation. 

## Setup simulation conditions

This sets up simulations conditions which is here a single pH-value and multiple salt concentrations.
The data is collected into a Pandas data-frame.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os.path
import os
import json
from jinja2 import Template
from itertools import product
import ruamel_yaml as yaml
from io import StringIO 

faunusdir = '/home/isabel/github/faunus/faunus' # linux
workdir = '/home/isabel/Documents/SI-betalac-lactoferrin-main/tempering/cosmos_v4/data_15seed' # linux

pH_range = [5.5]
salt_range = np.array([0.01, 0.015, 0.02, 0.025])# mol/l
offset = np.arange(70.0,126.0,1) # separations in Å
df = pd.DataFrame(product(pH_range, salt_range, offset), columns=['pH', 'Cs', 'offset'])
df.T

## Create simulation input files

Here we create input files where each salt concentration has its own directory.
The input files are created from `template.yml` using Jinja2.

In [5]:
%cd -q $workdir
def makeFaunusInput(outfile, **kwargs):
    ''' generate faunus input file from Jinja2 template '''
    with open('../template.yml', 'r') as file: # this is the template file
        template = Template(file.read())
        offset = kwargs['offset']
        dirname = f'Off{offset}'
        if not os.path.isdir(dirname):
            os.makedirs(dirname)
        with open(f'Off{offset}/' + outfile, 'w') as f:   # open a new faunus input file
            inputstr = template.render(kwargs)
            #print(inputstr)
            d = yaml.safe_load(StringIO(inputstr)) # convert string to stream
            f.write(yaml.dump(d))
    return dirname

In [6]:
# creating directories
dirnames = []
for idx, row in df.iterrows():
    dir = makeFaunusInput('eq.yml', Cs=row.Cs, pH=row.pH, micro=500, offset=row.offset)  # equilibration
    dir = makeFaunusInput('input.yml', Cs=row.Cs, pH=row.pH, micro=5000, offset=row.offset) # production
    dirnames.append(dir) # list with all names of directories

### Run simulations
The simulation is set up to run in parallel, with both an equilibration run and then production run. 

In [ ]:
# run simulations in parallel
import multiprocess as mp
numcpu=32
runeq=True
runprod= True
rerun = True

def launch_simulation(offset):
    if rerun or (not os.path.isfile('out.json')): # run only if there's no state file
        if runeq: # if state.json isn't there, run anyway
            !yason.py eq.yml | $faunusdir -v2 -o eqout.json 
        if runprod:
            print('.', end='')
            !yason.py input.yml | $faunusdir -v2 -s state.json

with mp.Pool(numcpu) as p: # split to multiple threads
    p.map(launch_simulation, offset)
print('done.')

## Plot potential of mean force, $w(R)$

For each condition, this:
1. Loads the sampled radial distribution function
2. Converts it to a free energy, $w^{\prime}(R)$
3. Shifts $w^{\prime}(R)$ to zero at large separations (finds $w_0$)
4. Plot

In [7]:

def get_forces(dirs):
    ''' extract force from out.json as a function of offset '''
    forces = []
    for dirname in dirs:
        if os.path.exists(dirname):
            #print(dirname)
            with open(dirname+'/out.json') as f:
                j = json.load(f) # --> dict
                if j['analysis'][0]["virtualtranslate"]["force"] is not None:
                    f = j['analysis'][0]["virtualtranslate"]["force"]
                    forces.append(-f)

    return forces #np.array([df.offset.values, forces])

def plotForce(ax, Cs_begin, Cs_end, style, label, Cs):    
    db = dirnames[Cs_begin:Cs_end]# filter database
    r = df.offset.values[Cs_begin:Cs_end]
    f = get_forces(db)
    w = np.cumsum(f)*np.diff(r)[:-2].mean()
    w = w - w[-1]
    ax.plot(r, w, style, label=label, lw=4, alpha=0.6)
    

In [ ]:
plt.rcParams.update({'font.size': 14, 'figure.figsize': [10, 13.0]})
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

plotForce(ax, 0, 36, style='r-', label='explicit 25mM', Cs= 25)
plotForce(ax, 36, 72, style='m-', label='explicit 50mM', Cs = 50)
plotForce(ax, 72, 109, style='k-', label='explicit 100mM', Cs = 100)

plt.xlim(45, 100)
plt.legend(loc=0, fontsize='small', frameon=False, ncol=2)
